In [ ]:
# VER TODO-LIST em README.md
from tools.reverso_tools import fetch_element_as_string, get_word_data
from reverso_context_api import Client
import json, re, string, random, time
import urllib.request
import spacy, nltk
import pandas as pd

In [2]:
TRANSLATION_NUMBER = 3
EXAMPLE_NUMBER = TRANSLATION_NUMBER
INPUT_LANGUAGE = "de", "Deutsch"
OUTPUT_LANGUAGE = "en", "English"
client = Client(INPUT_LANGUAGE[0], OUTPUT_LANGUAGE[0])

In [3]:
def get_text(file):
    f = open(file, "r", encoding="utf-8")
    text = f.read()
    tokens = nltk.word_tokenize(text)
    reworked_text = nltk.Text(tokens)

    return text, reworked_text

In [4]:
nlp = spacy.load("de_dep_news_trf")

In [5]:
text, wf = get_text("textos/creative.txt")

In [6]:
doc = nlp(text)

In [7]:
lemmas = [token.lemma_ for token in doc if not token.is_punct]
tokens = [token for token in doc if not token.is_punct]
pairs = list(set(zip(tokens, lemmas)))

In [8]:
# ways to perfect this function:
# check whether the first word in the concordance example is it
# if not, cut off this word (so as to remove incomplete words)
# do the same to the last word of the sentence

def boldify_selected_word(word, string):
    string_l = string.split()
    for index in range(len(string_l)):
        if string_l[index].lower() == word.lower():
            bold = "<b>"+word.lower()+"</b>"
            string_l[index] = bold 
    return string_l, bold

def get_context_and_clean_up(word, concordance):

    # get word context from concordance
    c = concordance.concordance_list((word))

    # split string and make chosen word bold
    line, bold_word = boldify_selected_word((word), c[0][6])
    word_list = line

    # remove first and last words of string, as they are sometimes incomplete
    # and assert that the first and last 
    if word_list[0] not in (word, bold_word):
        del word_list[0]
        if word_list[0] in string.punctuation:
            del word_list[0] 
    if word_list[len(word_list) - 1] not in (word, bold_word):
        del word_list[len(word_list) - 1]

    # get string with extra spaces
    unfinished_string = " ".join(word_list)
    
    # remove unnecessary spaces from string
    trailing_spaces_start = r"^\s+"
    trailing_spaces_end = r"\s+$"

    cleaned_up_pre = re.sub(r'\s([?.!,:;"](?:\s|$))', r'\1', unfinished_string)
    cleaned_up_start = re.sub(trailing_spaces_start, "", cleaned_up_pre)
    final_string = re.sub(trailing_spaces_end, "", cleaned_up_start)
    
    return final_string

def clean_up(string):
    trailing_spaces_start = r"^\s+"
    trailing_spaces_end = r"\s+$"

    cleaned_up_pre = re.sub(r'([?.!,:;"](?:\s|$))', "", string.split()[0])
    cleaned_up_start = re.sub(trailing_spaces_start, "", cleaned_up_pre)
    cleaned_up = re.sub(trailing_spaces_end, "", cleaned_up_start)

    return cleaned_up

In [ ]:
pre_material = []
for pair in pairs:
    pre_material.append((get_word_data(pair[1], 3, 3), get_context_and_clean_up(pair[0].text, wf)))
    print(pair[0].text)
    for i in range(0,5):
        time.sleep(1)
    time.sleep(random.random())

In [32]:
final_list = []
for i in range(len(c)-1):
    if c[i][0] != False:
        final_list.append(c[i])

In [38]:
for i in range(len(final_list) - 1):
    data = final_list[i][0]
    context = final_list[i][1]
    md = {
        "name":fetch_element_as_string(data,"name"),
        "translations": fetch_element_as_string(data,"translations"),
        "german": fetch_element_as_string(data, "input phrases"),
        "english": fetch_element_as_string(data, "output phrases"),
        "context": context
    }
    dataframe = pd.DataFrame.from_dict(md, orient="index")
    dataframe = dataframe.transpose()
    dataframe.to_csv("german_output.csv", mode="a", header=False, index=False)